# 小红书爆款文案生成助手（RAG 版）

本项目使用 DeepSeek LLM + Milvus 向量数据库 + RAG 技术，构建一个能够从 Markdown 产品文档中检索信息并生成小红书爆款文案的智能 Agent。

## 项目特点

- 产品数据来源于 `products.md` 文件
- 使用 Milvus 向量数据库存储产品信息
- 通过 RAG 技术实现语义检索
- 采用 ReAct 模式进行多轮推理

## 1. 环境准备

In [6]:
import os
import re
import json
import random
import time
from openai import OpenAI
from pymilvus import MilvusClient, model as milvus_model

# 初始化 DeepSeek 客户端
api_key = os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("请设置 DEEPSEEK_API_KEY 环境变量")

client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",
)

print("DeepSeek 客户端初始化完成")

DeepSeek 客户端初始化完成


## 2. 从 Markdown 文件加载产品数据

In [7]:
def load_products_from_markdown(file_path: str) -> list:
    """
    从 Markdown 文件加载产品数据。
    按二级标题（## ）分割，每个产品作为一条记录。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # 按二级标题分割
    sections = re.split(r'\n## ', content)
    
    products = []
    for section in sections[1:]:  # 跳过第一个（文件头部）
        lines = section.strip().split('\n')
        if lines:
            product_name = lines[0].strip()
            description = '\n'.join(lines[1:]).strip()
            products.append({
                'product_name': product_name,
                'description': f"产品名称：{product_name}\n{description}"
            })
    
    return products

# 加载产品数据
PRODUCTS = load_products_from_markdown('products.md')
print(f"已加载 {len(PRODUCTS)} 个产品：")
for p in PRODUCTS:
    print(f"  - {p['product_name']}")

已加载 8 个产品：
  - 深海蓝藻保湿面膜
  - 美白精华
  - 抗老精华
  - 水杨酸棉片
  - 防晒霜
  - 玻尿酸原液
  - 氨基酸洗面奶
  - 修护面霜


## 3. 构建向量数据库

In [8]:
# 初始化 Milvus 和 Embedding 模型
DB_PATH = "./product_database.db"
COLLECTION_NAME = "products"

from pymilvus import model as milvus_model

# OpenAI国内代理 https://api.apiyi.com/token 
embedding_model = milvus_model.dense.OpenAIEmbeddingFunction(
    model_name='text-embedding-3-large', # Specify the model name
    api_key='sk-6yR6RvofW3KRAOsp16B03e947cCb41C5A778Ee29Be2dFd32', # Provide your OpenAI API key
    base_url='https://api.apiyi.com/v1',
    dimensions=512
)

print(f"Milvus 数据库路径: {DB_PATH}")
print("Embedding 模型初始化完成")

Milvus 数据库路径: ./product_database.db
Embedding 模型初始化完成


In [9]:
# 创建 Collection 并导入数据
if milvus_client.has_collection(COLLECTION_NAME):
    milvus_client.drop_collection(COLLECTION_NAME)
    print(f"已删除旧的 collection: {COLLECTION_NAME}")

# 获取 embedding 维度
test_embedding = embedding_model.encode_queries(["测试"])[0]
embedding_dim = len(test_embedding)

# 创建 collection
milvus_client.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=embedding_dim,
    metric_type="COSINE",
    consistency_level="Strong",
)
print(f"已创建 collection: {COLLECTION_NAME}，维度: {embedding_dim}")

已创建 collection: products，维度: 512


In [10]:
# 生成向量并插入数据
descriptions = [p['description'] for p in PRODUCTS]
doc_embeddings = embedding_model.encode_documents(descriptions)

data_to_insert = []
for i, product in enumerate(PRODUCTS):
    data_to_insert.append({
        "id": i,
        "vector": doc_embeddings[i],
        "product_name": product['product_name'],
        "description": product['description']
    })

milvus_client.insert(collection_name=COLLECTION_NAME, data=data_to_insert)
print(f"已插入 {len(data_to_insert)} 条产品数据")

已插入 8 条产品数据


## 4. 定义 RAG 查询函数

In [11]:
def rag_query_product(product_name: str) -> str:
    """
    使用 RAG 技术查询产品数据库。
    """
    print(f"[RAG] 查询产品: {product_name}")
    
    try:
        query_embedding = embedding_model.encode_queries([product_name])
        
        results = milvus_client.search(
            collection_name=COLLECTION_NAME,
            data=query_embedding,
            limit=1,
            search_params={"metric_type": "COSINE", "params": {}},
            output_fields=["product_name", "description"]
        )
        
        if results and len(results[0]) > 0:
            match = results[0][0]
            similarity = match["distance"]
            matched_name = match["entity"]["product_name"]
            description = match["entity"]["description"]
            
            print(f"[RAG] 匹配: {matched_name}，相似度: {similarity:.4f}")
            
            if similarity < 0.3:
                return f"未找到关于 '{product_name}' 的产品信息。"
            
            return description
        else:
            return f"未找到关于 '{product_name}' 的产品信息。"
            
    except Exception as e:
        return f"查询出错: {str(e)}"

# 测试 RAG 查询
print("\n--- 测试 RAG 查询 ---")
test_result = rag_query_product("美白精华")
print(f"查询结果预览: {test_result[:200]}...")


--- 测试 RAG 查询 ---
[RAG] 查询产品: 美白精华
[RAG] 匹配: 美白精华，相似度: 0.6802
查询结果预览: 产品名称：美白精华
- **品牌**：SK-II
- **产品全称**：焕白光感精华液
- **核心成分**：烟酰胺 5%、VC 衍生物（抗坏血酸葡糖苷）、熊果苷
- **主要功效**：提亮肤色、淡化痘印、改善暗沉、均匀肤色
- **质地特点**：轻薄水润，上脸秒吸收，不油腻
- **适用肤质**：适合需要均匀肤色的人群，油皮干皮均可
- **规格**：30ml
- **使用方法**：早晚洁面爽肤...


## 5. 定义 Agent 组件

In [12]:
# System Prompt
SYSTEM_PROMPT = """
你是一个资深的小红书爆款文案专家，擅长结合最新潮流和产品卖点，创作引人入胜、高互动、高转化的笔记文案。

你的任务是根据用户提供的产品和需求，生成包含标题、正文、相关标签和表情符号的完整小红书笔记。

请始终采用'Thought-Action-Observation'模式进行推理和行动。文案风格需活泼、真诚、富有感染力。当完成任务后，请以JSON格式直接输出最终文案，格式如下：
```json
{
  "title": "小红书标题",
  "body": "小红书正文",
  "hashtags": ["#标签1", "#标签2", "#标签3", "#标签4", "#标签5"],
  "emojis": ["✨", "🔥", "💖"]
}
```
在生成文案前，请务必先思考并收集足够的信息。
"""

# Tools 定义
TOOLS_DEFINITION = [
    {
        "type": "function",
        "function": {
            "name": "search_web",
            "description": "搜索互联网上的实时信息，用于获取最新趋势、用户评价等。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "搜索关键词"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "query_product_database",
            "description": "查询产品数据库，获取产品的详细卖点、成分、适用人群等信息。",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_name": {"type": "string", "description": "产品名称"}
                },
                "required": ["product_name"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_emoji",
            "description": "根据文案内容生成适合小红书风格的表情符号。",
            "parameters": {
                "type": "object",
                "properties": {
                    "context": {"type": "string", "description": "文案关键内容"}
                },
                "required": ["context"]
            }
        }
    }
]

In [13]:
# 模拟工具函数
def mock_search_web(query: str) -> str:
    """模拟网页搜索"""
    print(f"[Tool] 搜索: {query}")
    time.sleep(0.5)
    if "美妆趋势" in query:
        return "近期小红书美妆流行'早C晚A'护肤理念、'伪素颜'妆容，热门关键词有#氛围感、#抗老、#屏障修复。"
    elif "保湿" in query or "面膜" in query:
        return "小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。"
    elif "抗老" in query:
        return "小红书抗老精华热门话题：早C晚A护肤法、视黄醇使用技巧。热门标签：#抗老精华 #抗初老"
    elif "美白" in query:
        return "小红书美白热门话题：烟酰胺美白、VC精华、冷白皮养成。"
    else:
        return f"关于 '{query}' 的市场反馈通常关注产品成分、功效和用户体验。"

def mock_generate_emoji(context: str) -> list:
    """模拟生成表情符号"""
    print(f"[Tool] 生成表情: {context}")
    emoji_map = {
        "补水": ["💦", "💧", "🌊", "✨"],
        "保湿": ["💦", "💧", "🌊", "✨"],
        "美白": ["✨", "🌟", "💫", "⭐"],
        "抗老": ["✨", "💎", "🌟", "💫", "👑"],
        "防晒": ["☀️", "🌞", "🧴", "✨"],
        "清洁": ["🫧", "💦", "✨", "🧼"],
    }
    for key, emojis in emoji_map.items():
        if key in context:
            return emojis
    return random.sample(["✨", "🔥", "💖", "💯", "🎉", "👍", "🤩"], k=4)

# 工具函数映射
available_tools = {
    "search_web": mock_search_web,
    "query_product_database": rag_query_product,  # 使用 RAG 查询
    "generate_emoji": mock_generate_emoji,
}

## 6. Agent 主函数

In [14]:
def generate_rednote(product_name: str, tone_style: str = "活泼甜美", max_iterations: int = 8) -> str:
    """
    使用 DeepSeek Agent 生成小红书爆款文案。
    """
    print(f"\n🚀 启动文案生成，产品：{product_name}，风格：{tone_style}\n")
    
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"请为产品「{product_name}」生成一篇小红书爆款文案。要求：语气{tone_style}，包含标题、正文、至少5个相关标签和5个表情符号。请以完整的JSON格式输出。"}
    ]
    
    for i in range(max_iterations):
        print(f"-- Iteration {i + 1} --")
        
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=messages,
                tools=TOOLS_DEFINITION,
                tool_choice="auto"
            )
            
            msg = response.choices[0].message
            
            if msg.tool_calls:
                print("Agent: 调用工具...")
                messages.append(msg)
                
                for tool_call in msg.tool_calls:
                    func_name = tool_call.function.name
                    func_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}
                    
                    print(f"  Action: {func_name}({func_args})")
                    
                    if func_name in available_tools:
                        result = available_tools[func_name](**func_args)
                        print(f"  Observation: {str(result)[:100]}...")
                        messages.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": str(result)
                        })
                    else:
                        messages.append({
                            "tool_call_id": tool_call.id,
                            "role": "tool",
                            "content": f"未知工具: {func_name}"
                        })
                        
            elif msg.content:
                # 尝试提取 JSON
                json_match = re.search(r"```json\s*(\{.*\})\s*```", msg.content, re.DOTALL)
                
                if json_match:
                    try:
                        result = json.loads(json_match.group(1))
                        print("✅ 文案生成完成！")
                        return json.dumps(result, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        messages.append(msg)
                else:
                    try:
                        result = json.loads(msg.content)
                        print("✅ 文案生成完成！")
                        return json.dumps(result, ensure_ascii=False, indent=2)
                    except json.JSONDecodeError:
                        messages.append(msg)
            else:
                break
                
        except Exception as e:
            print(f"错误: {e}")
            break
    
    print("\n⚠️ 未能生成最终文案")
    return "未能成功生成文案。"

## 7. 测试文案生成

In [15]:
# 测试 1: 深海蓝藻保湿面膜
result_1 = generate_rednote("深海蓝藻保湿面膜", "活泼甜美")
print("\n--- 生成的文案 ---")
print(result_1)


🚀 启动文案生成，产品：深海蓝藻保湿面膜，风格：活泼甜美

-- Iteration 1 --
Agent: 调用工具...
  Action: query_product_database({'product_name': '深海蓝藻保湿面膜'})
[RAG] 查询产品: 深海蓝藻保湿面膜
[RAG] 匹配: 深海蓝藻保湿面膜，相似度: 0.7913
  Observation: 产品名称：深海蓝藻保湿面膜
- **品牌**：海蓝之谜
- **核心成分**：深海蓝藻提取物，富含多糖和氨基酸
- **主要功效**：深层补水、修护肌肤屏障、舒缓敏感泛红
- **质地特点**：清爽不...
-- Iteration 2 --
Agent: 调用工具...
  Action: search_web({'query': '2024小红书面膜护肤趋势 保湿面膜推荐'})
[Tool] 搜索: 2024小红书面膜护肤趋势 保湿面膜推荐
  Observation: 小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。...
-- Iteration 3 --
Agent: 调用工具...
  Action: search_web({'query': '深海蓝藻面膜成分功效 用户评价'})
[Tool] 搜索: 深海蓝藻面膜成分功效 用户评价
  Observation: 小红书保湿面膜热门话题：沙漠干皮救星、熬夜急救面膜、水光肌养成。...
-- Iteration 4 --
Agent: 调用工具...
  Action: generate_emoji({'context': '深海蓝藻保湿面膜 补水修护 敏感肌友好 水光肌 沙漠干皮救星'})
[Tool] 生成表情: 深海蓝藻保湿面膜 补水修护 敏感肌友好 水光肌 沙漠干皮救星
  Observation: ['💦', '💧', '🌊', '✨']...
-- Iteration 5 --
✅ 文案生成完成！

--- 生成的文案 ---
{
  "title": "沙漠干皮救星！深海蓝藻面膜让我get水光肌✨",
  "body": "姐妹们！最近发现了一款宝藏面膜！海蓝之谜的深海蓝藻保湿面膜真的绝了！💖\n\n作为一个干敏皮，一到换季就各种泛红起皮，这款面膜简直就是我的救星！🌊\n\n🌟核心成分：深海蓝藻

In [16]:
# 测试 2: 美白精华
result_2 = generate_rednote("美白精华", "知性温柔")
print("\n--- 生成的文案 ---")
print(result_2)


🚀 启动文案生成，产品：美白精华，风格：知性温柔

-- Iteration 1 --
Agent: 调用工具...
  Action: search_web({'query': '2024美白精华最新趋势 小红书热门话题'})
[Tool] 搜索: 2024美白精华最新趋势 小红书热门话题
  Observation: 小红书美白热门话题：烟酰胺美白、VC精华、冷白皮养成。...
-- Iteration 2 --
Agent: 调用工具...
  Action: query_product_database({'product_name': '美白精华'})
[RAG] 查询产品: 美白精华
[RAG] 匹配: 美白精华，相似度: 0.6802
  Observation: 产品名称：美白精华
- **品牌**：SK-II
- **产品全称**：焕白光感精华液
- **核心成分**：烟酰胺 5%、VC 衍生物（抗坏血酸葡糖苷）、熊果苷
- **主要功效**：提亮肤色、淡化...
-- Iteration 3 --
Agent: 调用工具...
  Action: search_web({'query': '美白精华真实使用感受 小红书用户评价'})
[Tool] 搜索: 美白精华真实使用感受 小红书用户评价
  Observation: 小红书美白热门话题：烟酰胺美白、VC精华、冷白皮养成。...
-- Iteration 4 --
Agent: 调用工具...
  Action: generate_emoji({'context': '美白精华 提亮肤色 淡斑 护肤 温柔知性 冷白皮'})
[Tool] 生成表情: 美白精华 提亮肤色 淡斑 护肤 温柔知性 冷白皮
  Observation: ['✨', '🌟', '💫', '⭐']...
-- Iteration 5 --
✅ 文案生成完成！

--- 生成的文案 ---
{
  "title": "温柔变白记｜这瓶精华让我从暗沉到透亮✨",
  "body": "姐妹们，今天想和你们分享我的美白心路历程💫\n\n曾经的我也是暗沉肌代表，肤色不均、痘印明显，拍照总是要开美颜。直到遇到了这瓶美白精华，我的护肤世界彻底改变了🌟\n\n✨为什么选择它？\n- 核心成分很能打：5%烟酰胺+VC衍生物+熊果苷，三重

In [17]:
# 测试 3: 氨基酸洗面奶（新增产品）
result_3 = generate_rednote("氨基酸洗面奶", "专业理性")
print("\n--- 生成的文案 ---")
print(result_3)


🚀 启动文案生成，产品：氨基酸洗面奶，风格：专业理性

-- Iteration 1 --
Agent: 调用工具...
  Action: search_web({'query': '氨基酸洗面奶 2024 最新趋势 用户评价'})
[Tool] 搜索: 氨基酸洗面奶 2024 最新趋势 用户评价
  Observation: 关于 '氨基酸洗面奶 2024 最新趋势 用户评价' 的市场反馈通常关注产品成分、功效和用户体验。...
-- Iteration 2 --
Agent: 调用工具...
  Action: query_product_database({'product_name': '氨基酸洗面奶'})
[RAG] 查询产品: 氨基酸洗面奶
[RAG] 匹配: 氨基酸洗面奶，相似度: 0.7717
  Observation: 产品名称：氨基酸洗面奶
- **品牌**：芙丽芳丝
- **产品全称**：净润洗面霜
- **核心成分**：氨基酸表活、甘草酸二钾、透明质酸
- **主要功效**：温和清洁、保湿不紧绷、维护肌肤屏障
...
-- Iteration 3 --
Agent: 调用工具...
  Action: search_web({'query': '氨基酸洗面奶 成分分析 温和清洁 护肤原理'})
[Tool] 搜索: 氨基酸洗面奶 成分分析 温和清洁 护肤原理
  Observation: 关于 '氨基酸洗面奶 成分分析 温和清洁 护肤原理' 的市场反馈通常关注产品成分、功效和用户体验。...
-- Iteration 4 --
Agent: 调用工具...
  Action: generate_emoji({'context': '氨基酸洗面奶 温和清洁 保湿不紧绷 敏感肌适用 护肤成分分析'})
[Tool] 生成表情: 氨基酸洗面奶 温和清洁 保湿不紧绷 敏感肌适用 护肤成分分析
  Observation: ['💦', '💧', '🌊', '✨']...
-- Iteration 5 --
✅ 文案生成完成！

--- 生成的文案 ---
{
  "title": "氨基酸洗面奶成分深度解析｜温和清洁的护肤科学",
  "body": "作为护肤博主，今天从专业角度分析氨基酸洗面奶的护肤原理和成分优势。\n\n🔬【核心成分解析】\

## 8. 格式化输出

In [18]:
def format_for_rednote(json_string: str) -> str:
    """将 JSON 格式的文案转换为小红书发布格式"""
    try:
        data = json.loads(json_string)
    except json.JSONDecodeError as e:
        return f"解析失败: {e}"
    
    title = data.get("title", "无标题")
    body = data.get("body", "")
    hashtags = data.get("hashtags", [])
    
    output = f"## {title}\n\n{body}\n\n"
    if hashtags:
        output += " ".join(hashtags)
    
    return output.strip()

# 格式化输出
print("--- 小红书发布格式 ---")
print(format_for_rednote(result_1))

--- 小红书发布格式 ---
## 沙漠干皮救星！深海蓝藻面膜让我get水光肌✨

姐妹们！最近发现了一款宝藏面膜！海蓝之谜的深海蓝藻保湿面膜真的绝了！💖

作为一个干敏皮，一到换季就各种泛红起皮，这款面膜简直就是我的救星！🌊

🌟核心成分：深海蓝藻提取物，富含多糖和氨基酸，深层补水的同时还能修护肌肤屏障！

💦使用感受：
1. 精华液超多！一片足足25ml，敷完脸还能涂脖子和手臂
2. 质地清爽不粘腻，吸收超快，敷完不用洗直接睡觉
3. 敷完皮肤水水嫩嫩，像喝饱了水一样，第二天上妆超服帖

✨适合人群：
- 沙漠大干皮
- 敏感肌姐妹
- 熬夜党急救
- 想要水光肌的宝子们

价格也很亲民，168元/盒5片，性价比超高！已经回购第二盒了！

#护肤分享 #好物推荐 #一起敷面膜

#深海蓝藻保湿面膜 #沙漠干皮救星 #水光肌养成 #敏感肌友好 #熬夜急救面膜 #海蓝之谜 #保湿面膜推荐


## 9. 总结

本项目实现了一个完整的 RAG 文案生成流程：

1. **数据来源**：`products.md` Markdown 文件
2. **向量存储**：`product_database.db` Milvus 数据库
3. **检索方式**：语义相似度搜索（RAG）
4. **生成模式**：ReAct (Thought-Action-Observation)

### 扩展方向

- 添加更多产品到 `products.md`
- 接入真实的网络搜索 API
- 添加敏感词过滤工具
- 支持多模态（图片+文案）生成